In [2]:
import pandas as pd
import json

In [3]:
exp_num_un_countries = 232

In [4]:
df = pd.read_excel('./UN_MigrantStockByOriginAndDestination_2019.xlsx', sheet_name='Table 1')
df = df.rename(columns={'Unnamed: 0': 'year', 'Unnamed: 2': 'dest', 'Unnamed: 5': 'type_of_data', 'Unnamed: 6': 'total'})

cdf = df.iloc[14:15, 9:]
cdf = cdf.dropna(axis=1)

em_total_df = df.iloc[1713:1714, :]
df = df[16:]
df = df[df['type_of_data'].notnull()]

In [5]:
orig_countries = cdf.values.flatten().tolist()
assert len(orig_countries) == exp_num_un_countries

In [6]:
years = [1990, 1995, 2000, 2005, 2010, 2015, 2019]
dest_countries = {}
for y in years:
    dest_countries[y] = df[df['year'] == y]['dest'].values.tolist()

for k in dest_countries:
    assert len(dest_countries[k]) == exp_num_un_countries

In [7]:
rename = {}
for i, oc in enumerate(orig_countries):
    rename[f'Unnamed: {i + 9}'] = oc
em_total_df = em_total_df.rename(columns=rename)
df = df.rename(columns=rename)
em_total_df = em_total_df[em_total_df.columns[~(em_total_df.columns.str.contains('Unnamed') | em_total_df.columns.str.contains('type_of_data'))]]
df = df[df.columns[~(df.columns.str.contains('Unnamed') | df.columns.str.contains('type_of_data'))]]

In [19]:
mdf = pd.read_csv('./mapping.csv')
mapping = {}
mapping_rev = {}
for i in range(0, len(mdf)):
    mapping[mdf.iloc[i, 0]] = mdf.iloc[i, 1]
    mapping_rev[mdf.iloc[i, 1]] = mdf.iloc[i, 0]

d2 = {
    "Falkland Islands (Malvinas)": "fk",
    "Iran (Islamic Republic of)": "ir",
    "Lao People's Democratic Republic": "la",
    "Republic of Korea": "kr",
    "Dem. People's Republic of Korea": "kp",
    "United Republic of Tanzania": "tz",
    "Bolivia (Plurinational State of)": "bo",
    "Russian Federation": "ru",
    "China, Hong Kong SAR": "hk",
    "China, Macao SAR": "mo",
    "Republic of Moldova": "md",
    "State of Palestine": "ps",
    "Syrian Arab Republic": "sy",
    "Venezuela (Bolivarian Republic of)": "ve",
    "Cabo Verde": "cv"
}

skip = [
    # can't determine country code from 'Channel Islands', could be Jersey or Guernsey
    "Channel Islands"
]

def id_of_un_country_string(s):
    if s in mapping_rev:
        return mapping_rev[s]
    elif s in d2:
        return d2[s]
    elif s in skip:
        return None
    else:
        raise ValueError(f"Unknown country string: {s}")

all_country_ids = [id_of_un_country_string(c) for c in orig_countries]
all_country_ids = [x for x in all_country_ids if x != None]

assert len(set(all_country_ids)) == len(orig_countries) - len(skip)

In [26]:
id_of = {c : id_of_un_country_string(c) for c in orig_countries if id_of_un_country_string(c) is not None}
of_id = {id_of_un_country_string(c): c for c in orig_countries if id_of_un_country_string(c) is not None}


with open('un_country.py', 'w') as f:
    print(f"id_of_string = {id_of}", file=f)
    print(f"string_of_id = {of_id}", file=f)

In [9]:
df_2019 = df[df.year == 2019]

In [10]:
def insert(d, k1, k2, k3, v):
    if k1 in ["be", "mc"] and k2 in ["be", "mc"]:
        print(k1, k2)
    if k1 in d:
        if k2 in d[k1]:
            if k3 in d[k1][k2]:
                d[k1][k2][k3] += v
            else:
                d[k1][k2][k3] = v
        else:
            d[k1][k2] = {k3: v}
    else:
        d[k1] = {k2: {k3: v}}

In [11]:
def insert(d, keys, v):
    d_ = d
    for k in keys[:-1]:
        if k in d_:
            d_ = d_[k]
        else:
            d_[k] = {}
            d_ = d_[k]
    lk = keys[-1]
    if lk in d_:
        d_[lk] += v
    else:
        d_[lk] = v
        
d = {}
insert(d, ['gb', 'au', 'im'], 10)
insert(d, ['au', 'gb', 'em'], 10)
insert(d, ['gb', 'au', 'em'], 10)
insert(d, ['gb', 'im_tot'], 1000)
print(d)


{'gb': {'au': {'im': 10, 'em': 10}, 'im_tot': 1000}, 'au': {'gb': {'em': 10}}}


In [12]:
d_2019 = {}

dest_ids = set()
for _, r in df_2019.iterrows():
    dcid = id_of_un_country_string(r['dest'])
    dest_ids.add(dcid)
    
assert len(dest_ids) == len(df_2019)
assert len(orig_countries) == len(df_2019)

for _, r in df_2019.iterrows():
    if r['dest'] in skip:
        continue
    dcid = id_of_un_country_string(r['dest'])
    insert(d_2019, [dcid, 'im_tot'], r['total'])
    insert(d_2019, [dcid, 'em_tot'], em_total_df[r['dest']].iloc[0])
    for oc in orig_countries:
        if oc in skip or oc == r['dest']:
            continue
        ocid = id_of_un_country_string(oc)
        v = r[oc]

        if v == v: # not NaN
            # im = immigrants
            # em = emigrants
            #
            # if a person emigrates from A to B,
            # then they are an immigrant in B, from A
            insert(d_2019, [dcid, ocid, 'im'], v)
            insert(d_2019, [ocid, dcid, 'em'], v)

In [13]:
for cid in d_2019:
    d = d_2019[cid]
    total_immigrants = d['im_tot']
    sum_immigrants_known_origin = sum(d[ocid].get('im', 0) for ocid in d if ocid not in ['im_tot', 'em_tot'])
    # ideally this would be an equality, but unfortunately for lots of countries we don't know
    # where some of their immigrants came from
    assert sum_immigrants_known_origin <= total_immigrants
    
ems = 0
ims = 0
for cid in d_2019:
    for cid_ in d_2019[cid]:
        if len(cid_) != 2:
            continue
        ims += d_2019[cid][cid_].get('im', 0)
        ems += d_2019[cid][cid_].get('em', 0)
assert ems == ims, f"{ems} vs {ims}"

ems = 0
ims = 0
for cid in d_2019:
    ems += d_2019[cid]['em_tot']
    ims += d_2019[cid]['im_tot']
# number of immigrants should equal number of emigrants, but it does not
assert ems != ims, f"{ems} vs {ims}, diff={ims - ems}"       

In [29]:
s = json.dumps(d_2019, indent=4)
with open('d_2019.js', 'w', encoding='utf-8') as f:
    f.write(f"export const d = {s};")
with open('d_2019.json', 'w', encoding='utf-8') as f:
    f.write(s)